In [14]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

## [MY CODE] Set Tokenizer
- Hugging Face의 Transformer 아키텍처 기반의 Distilbert 모델 사용
- distilbert는 knowledge distillation 기법을 사용하여 Bert 모델 대비 성능은 유지하면서 속도와 크기를 줄인 경량화 모델
- knowledge distillation 기법은 Teacher model / Student model을 이용하여 훈련시키는 기법
- uncased 모델은 텍스트 입력에서 대소문자 구분은 수행하지 않음

In [15]:
import torch
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader

# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

## [MY CODE] Load Dataset
- Huggingface dataset의 fancyzhx/ag_news을 사용
- trainset volume: 120,000
- testset volume: 7,600

In [16]:
ds = load_dataset("fancyzhx/ag_news")
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

## [MY CODE] define collate_fn
- collate_fn은 pytorch dataloader에서 사용하는 함수로, 배치 데이터를 전처리하여 모델에 입력할 수 있는 형태로 변환하는 역할을 수행
- 뉴스의 전반적인 내용을 기반으로 분류를 수행해야 하기 때문에, truncation을 disabled 처리.

In [17]:
def collate_fn(batch):
  # 토큰화된 텍스트의 최대 길이를 400으로 제한할 수 있도록 변수 값 설정. 초과되면 자르고(truncation), 부족하면 패딩할 수 있음(padding)
  max_len = 400
  texts, labels = [], []

  # 준비된 데이터셋의 features: ['text', 'label'] 이므로 이에 맞게 분류
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  # 데이터셋에서 추출한 text, label 특성 별 tensor 형태로 변환
  """
  torch.LongTensor(): 입력 값이나 토큰화된 결과를 PyTorch의 텐서로 변환

  tokenizer(): 텍스트를 토큰화하는 함수
  - padding=True: 배치 내 가장 긴 시퀀스에 맞춰 작업, 남는 길이는 패딩 처리
  - truncation=True: 최대 길이를 초과하면 자르도록 수행 (-> 과제에서 관련 부분 지우라고 했는데, 확인 필요)
  - max_length=max_len: 시퀀스의 최대 길이를 max_len값(예: 400) 만큼 제한
  - .input_ids: 토크나이저가 반환한 입력 ID(=숫자로 변환된 텍스트)를 가져옴
    - (참고) 텍스트가 토큰화 된 결과 예시: [input_ids, attention_mask, ...]
  """
  # texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  texts = tokenizer(texts, padding=True, return_tensors='pt').input_ids
  labels = torch.LongTensor(labels)

  return texts, labels

## [MY CODE] Define dataloader
- pytorch의 data loader는 데이터셋을 배치 단위로 나누고 배치를 반복 가능한 형태로 제공
- trainset과 testset을 구분하여 data loader를 정의
- batch size (정수값): 데이터셋의 배치 사이즈를 어느 크기로 나눌지 설정 (기존:64 -> 변경:512)
- shuffle (True/False): 데이터셋 샘플링 순서의 무작위 여부 설정
- collate_fn : 사용자 정의 전처리 방법 입력

In [19]:
train_loader = DataLoader(
    ds['train'], batch_size=512, shuffle=True, collate_fn=collate_fn
)

test_loader = DataLoader(
    ds['test'], batch_size=512, shuffle=False, collate_fn=collate_fn
)

## [MY CODE] load Model
- 실제 분류를 수행할 Distilbert 모델 로드
- 특히, classification을 수행할 모델을 로드(label은 dataset의 classes 참고)
  - AutoModel.from_pretrained()로 로드하면 bert모델의 output이 마지막 레이어의 히든 스테이트를 반환
  - 따라서, 최종적으로 분류 작업을 수행하려면 추가적인 **헤드(head)**를 수동으로 추가해야 함.

In [20]:
from transformers import AutoModel, AutoModelForSequenceClassification

# model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4) # World, Sports, Business, Sci/Tech
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [10]:
model = AutoModel.from_pretrained("distilbert-base-uncased")
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [21]:
from torch import nn

class TextClassifier(nn.Module):
  def __init__(self, classes):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    # self.encoder = AutoModel.from_pretrained("distilbert-base-uncased")
    self.classifier = nn.Linear(768, classes)

  def forward(self, x):
    x = self.encoder(x)['last_hidden_state']
    x = self.classifier(x[:, 0])

    return x

model = TextClassifier(4)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


## Classifier Layer를 제외한 나머지 부분 freeze

In [22]:
for param in model.encoder.parameters():
  param.requires_grad = False

> 위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.

> 즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

In [ ]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0] # model : DistilBERT
    loss = loss_fn(preds, labels) # loss_fn = nn.CrossEntropyLoss()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 711862.5481872559
Epoch   1 | Train Loss: 710009.7866516113
Epoch   2 | Train Loss: 709800.1779174805
Epoch   3 | Train Loss: 709745.4475860596
Epoch   4 | Train Loss: 709840.5440826416
Epoch   5 | Train Loss: 709731.3714141846


In [ ]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1) # for classification
    # preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  # train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  # print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")
  print(f"Test acc: {test_acc:.3f}")